## LangGraph Open Deep Research - Supervisor-Researcher Architecture

In this notebook, we'll explore the **supervisor-researcher delegation architecture** for conducting deep research with LangGraph.

You can visit this repository to see the original application: [Open Deep Research](https://github.com/langchain-ai/open_deep_research)

Let's jump in!

## What We're Building

This implementation uses a **hierarchical delegation pattern** where:

1. **User Clarification** - Optionally asks clarifying questions to understand the research scope
2. **Research Brief Generation** - Transforms user messages into a structured research brief
3. **Supervisor** - A lead researcher that analyzes the brief and delegates research tasks
4. **Parallel Researchers** - Multiple sub-agents that conduct focused research simultaneously
5. **Research Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings are combined into a comprehensive report

![Architecture Diagram](https://i.imgur.com/Q8HEZn0.png)

This differs from a section-based approach by allowing dynamic task decomposition based on the research question, rather than predefined sections.

---

# 🤝 Breakout Room #1
## Deep Research Foundations

In this breakout room, we'll understand the architecture and components of the Open Deep Research system.

## Task 1: Dependencies

You'll need API keys for Anthropic (for the LLM) and Tavily (for web search). We'll configure the system to use Anthropic's Claude Sonnet 4 exclusively.

In [1]:
import os
import getpass

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

## Task 2: State Definitions

The state structure is hierarchical with three levels:

### Agent State (Top Level)
Contains the overall conversation messages, research brief, accumulated notes, and final report.

### Supervisor State (Middle Level)
Manages the research supervisor's messages, research iterations, and coordinating parallel researchers.

### Researcher State (Bottom Level)
Each individual researcher has their own message history, tool call iterations, and research findings.

We also have structured outputs for tool calling:
- **ConductResearch** - Tool for supervisor to delegate research to a sub-agent
- **ResearchComplete** - Tool to signal research phase is done
- **ClarifyWithUser** - Structured output for asking clarifying questions
- **ResearchQuestion** - Structured output for the research brief

Let's import these from our library: [`open_deep_library/state.py`](open_deep_library/state.py)

In [2]:
# Import state definitions from the library
from open_deep_library.state import (
    # Main workflow states
    AgentState,           # Lines 65-72: Top-level agent state with messages, research_brief, notes, final_report
    AgentInputState,      # Lines 62-63: Input state is just messages
    
    # Supervisor states
    SupervisorState,      # Lines 74-81: Supervisor manages research delegation and iterations
    
    # Researcher states
    ResearcherState,      # Lines 83-90: Individual researcher with messages and tool iterations
    ResearcherOutputState, # Lines 92-96: Output from researcher (compressed research + raw notes)
    
    # Structured outputs for tool calling
    ConductResearch,      # Lines 15-19: Tool for delegating research to sub-agents
    ResearchComplete,     # Lines 21-22: Tool to signal research completion
    ClarifyWithUser,      # Lines 30-41: Structured output for user clarification
    ResearchQuestion,     # Lines 43-48: Structured output for research brief
)

## Task 3: Utility Functions and Tools

The system uses several key utilities:

### Search Tools
- **tavily_search** - Async web search with automatic summarization to stay within token limits
- Supports Anthropic native web search and Tavily API

### Reflection Tools
- **think_tool** - Allows researchers to reflect on their progress and plan next steps (ReAct pattern)

### Helper Utilities
- **get_all_tools** - Assembles the complete toolkit (search + MCP + reflection)
- **get_today_str** - Provides current date context for research
- Token limit handling utilities for graceful degradation

These are defined in [`open_deep_library/utils.py`](open_deep_library/utils.py)

In [3]:
# Import utility functions and tools from the library
from open_deep_library.utils import (
    # Search tool - Lines 43-136: Tavily search with automatic summarization
    tavily_search,
    
    # Reflection tool - Lines 219-244: Strategic thinking tool for ReAct pattern
    think_tool,
    
    # Tool assembly - Lines 569-597: Get all configured tools
    get_all_tools,
    
    # Date utility - Lines 872-879: Get formatted current date
    get_today_str,
    
    # Supporting utilities for error handling
    get_api_key_for_model,          # Lines 892-914: Get API keys from config or env
    is_token_limit_exceeded,         # Lines 665-701: Detect token limit errors
    get_model_token_limit,           # Lines 831-846: Look up model's token limit
    remove_up_to_last_ai_message,    # Lines 848-866: Truncate messages for retry
    anthropic_websearch_called,      # Lines 607-637: Detect Anthropic native search usage
    openai_websearch_called,         # Lines 639-658: Detect OpenAI native search usage
    get_notes_from_tool_calls,       # Lines 599-601: Extract notes from tool messages
)

## Task 4: Configuration System

The configuration system controls:

### Research Behavior
- **allow_clarification** - Whether to ask clarifying questions before research
- **max_concurrent_research_units** - How many parallel researchers can run (default: 5)
- **max_researcher_iterations** - How many times supervisor can delegate research (default: 6)
- **max_react_tool_calls** - Tool call limit per researcher (default: 10)

### Model Configuration
- **research_model** - Model for research and supervision (we'll use Anthropic)
- **compression_model** - Model for synthesizing findings
- **final_report_model** - Model for writing the final report
- **summarization_model** - Model for summarizing web search results

### Search Configuration
- **search_api** - Which search API to use (ANTHROPIC, TAVILY, or NONE)
- **max_content_length** - Character limit before summarization

Defined in [`open_deep_library/configuration.py`](open_deep_library/configuration.py)

In [4]:
# Import configuration from the library
from open_deep_library.configuration import (
    Configuration,    # Lines 38-247: Main configuration class with all settings
    SearchAPI,        # Lines 11-17: Enum for search API options (ANTHROPIC, TAVILY, NONE)
)

## Task 5: Prompt Templates

The system uses carefully engineered prompts for each phase:

### Phase 1: Clarification
**clarify_with_user_instructions** - Analyzes if the research scope is clear or needs clarification

### Phase 2: Research Brief
**transform_messages_into_research_topic_prompt** - Converts user messages into a detailed research brief

### Phase 3: Supervisor
**lead_researcher_prompt** - System prompt for the supervisor that manages delegation strategy

### Phase 4: Researcher
**research_system_prompt** - System prompt for individual researchers conducting focused research

### Phase 5: Compression
**compress_research_system_prompt** - Prompt for synthesizing research findings without losing information

### Phase 6: Final Report
**final_report_generation_prompt** - Comprehensive prompt for writing the final report

All prompts are defined in [`open_deep_library/prompts.py`](open_deep_library/prompts.py)

In [5]:
# Import prompt templates from the library
from open_deep_library.prompts import (
    clarify_with_user_instructions,                    # Lines 3-41: Ask clarifying questions
    transform_messages_into_research_topic_prompt,     # Lines 44-77: Generate research brief
    lead_researcher_prompt,                            # Lines 79-136: Supervisor system prompt
    research_system_prompt,                            # Lines 138-183: Researcher system prompt
    compress_research_system_prompt,                   # Lines 186-222: Research compression prompt
    final_report_generation_prompt,                    # Lines 228-308: Final report generation
)

## ❓ Question #1:

Explain the interrelationships between the three states (Agent, Supervisor, Researcher). Why don't we just make a single huge state?

##### Answer:
Interrelationships between the three states (Agent, Supervisor, Researcher)
- Agent State: Holds the entire conversation history, the final research brief, accumulated notes from all researchers, and the final generated report.
- Supervisor State: Manages the "Lead Researcher's" thoughts, tracks how many research iterations have occurred, and coordinates the delegation of tasks.
- Researcher State: Tracks the specific search queries, tool outputs (ReAct pattern), and findings for a single, narrow research topic.

Why don't we just make a single huge state?
- Enables parallelization without overwriting states from different states
- Token limits & avoids context overlead
- Seperation of concerns - ensure the LLM isn't distracted by irrelevant details from other branches of the research
- Error Isolations 



## ❓ Question #2:

What are the advantages and disadvantages of importing these components instead of including them in the notebook?

##### Answer:
Modularity - easily reused across multiple different research notebooks without duplicating code


## 🏗️ Activity #1: Explore the Prompts

Open `open_deep_library/prompts.py` and examine one of the prompt templates in detail.

**Requirements:**
1. Choose one prompt template (clarify, brief, supervisor, researcher, compression, or final report)
2. Explain what the prompt is designed to accomplish
3. Identify 2-3 key techniques used in the prompt (e.g., structured output, role definition, examples)
4. Suggest one improvement you might make to the prompt

**YOUR CODE HERE** - Write your analysis in a markdown cell below

Analyis of transform_messages_into_research_topic_prompt

designed to accomplish
- primary goal is to convert a history of messages into a single, highly specific research question that acts as the dierction for the supervisor and parallel researchers.

techniques used
- First-Person Perspective Modeling - helps maintain alignment with user's original intent
- Domain source prioritizaiton - "if-then" logic for different research types. For example, it directs the model to prioritize official brand sites or Amazon for product research, while favoring original papers or official journals for academic queries
- Avoid Unwarranted Assumptions

what to improve
- Explicit bad sources




---

# 🤝 Breakout Room #2
## Building & Running the Researcher

In this breakout room, we'll explore the node functions, build the graph, and run wellness research.

## Task 6: Node Functions - The Building Blocks

Now let's look at the node functions that make up our graph. We'll import them from the library and understand what each does.

### The Complete Research Workflow

The workflow consists of 8 key nodes organized into 3 subgraphs:

1. **Main Graph Nodes:**
   - `clarify_with_user` - Entry point that checks if clarification is needed
   - `write_research_brief` - Transforms user input into structured research brief
   - `final_report_generation` - Synthesizes all research into final report

2. **Supervisor Subgraph Nodes:**
   - `supervisor` - Lead researcher that plans and delegates
   - `supervisor_tools` - Executes supervisor's tool calls (delegation, reflection)

3. **Researcher Subgraph Nodes:**
   - `researcher` - Individual researcher conducting focused research
   - `researcher_tools` - Executes researcher's tool calls (search, reflection)
   - `compress_research` - Synthesizes researcher's findings

All nodes are defined in [`open_deep_library/deep_researcher.py`](open_deep_library/deep_researcher.py)

### Node 1: clarify_with_user

**Purpose:** Analyzes user messages and asks clarifying questions if the research scope is unclear.

**Key Steps:**
1. Check if clarification is enabled in configuration
2. Use structured output to analyze if clarification is needed
3. If needed, end with a clarifying question for the user
4. If not needed, proceed to research brief with verification message

**Implementation:** [`open_deep_library/deep_researcher.py` lines 60-115](open_deep_library/deep_researcher.py#L60-L115)

In [28]:
# Import the clarify_with_user node
from open_deep_library.deep_researcher import clarify_with_user

### Node 2: write_research_brief

**Purpose:** Transforms user messages into a structured research brief for the supervisor.

**Key Steps:**
1. Use structured output to generate detailed research brief from messages
2. Initialize supervisor with system prompt and research brief
3. Set up supervisor messages with proper context

**Why this matters:** A well-structured research brief helps the supervisor make better delegation decisions.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 118-175](open_deep_library/deep_researcher.py#L118-L175)

In [29]:
# Import the write_research_brief node
from open_deep_library.deep_researcher import write_research_brief

### Node 3: supervisor

**Purpose:** Lead research supervisor that plans research strategy and delegates to sub-researchers.

**Key Steps:**
1. Configure model with three tools:
   - `ConductResearch` - Delegate research to a sub-agent
   - `ResearchComplete` - Signal that research is done
   - `think_tool` - Strategic reflection before decisions
2. Generate response based on current context
3. Increment research iteration count
4. Proceed to tool execution

**Decision Making:** The supervisor uses `think_tool` to reflect before delegating research, ensuring thoughtful decomposition of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 178-223](open_deep_library/deep_researcher.py#L178-L223)

In [30]:
# Import the supervisor node (from supervisor subgraph)
from open_deep_library.deep_researcher import supervisor

### Node 4: supervisor_tools

**Purpose:** Executes the supervisor's tool calls, including strategic thinking and research delegation.

**Key Steps:**
1. Check exit conditions:
   - Exceeded maximum iterations
   - No tool calls made
   - `ResearchComplete` called
2. Process `think_tool` calls for strategic reflection
3. Execute `ConductResearch` calls in parallel:
   - Spawn researcher subgraphs for each delegation
   - Limit to `max_concurrent_research_units` (default: 5)
   - Gather all results asynchronously
4. Aggregate findings and return to supervisor

**Parallel Execution:** This is where the magic happens - multiple researchers work simultaneously on different aspects of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 225-349](open_deep_library/deep_researcher.py#L225-L349)

In [31]:
# Import the supervisor_tools node
from open_deep_library.deep_researcher import supervisor_tools

### Node 5: researcher

**Purpose:** Individual researcher that conducts focused research on a specific topic.

**Key Steps:**
1. Load all available tools (search, MCP, reflection)
2. Configure model with tools and researcher system prompt
3. Generate response with tool calls
4. Increment tool call iteration count

**ReAct Pattern:** Researchers use `think_tool` to reflect after each search, deciding whether to continue or provide their answer.

**Available Tools:**
- Search tools (Tavily or Anthropic native search)
- `think_tool` for strategic reflection
- `ResearchComplete` to signal completion
- MCP tools (if configured)

**Implementation:** [`open_deep_library/deep_researcher.py` lines 365-424](open_deep_library/deep_researcher.py#L365-L424)

In [32]:
# Import the researcher node (from researcher subgraph)
from open_deep_library.deep_researcher import researcher

### Node 6: researcher_tools

**Purpose:** Executes the researcher's tool calls, including searches and strategic reflection.

**Key Steps:**
1. Check early exit conditions (no tool calls, native search used)
2. Execute all tool calls in parallel:
   - Search tools fetch and summarize web content
   - `think_tool` records strategic reflections
   - MCP tools execute external integrations
3. Check late exit conditions:
   - Exceeded `max_react_tool_calls` (default: 10)
   - `ResearchComplete` called
4. Continue research loop or proceed to compression

**Error Handling:** Safely handles tool execution errors and continues with available results.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 435-509](open_deep_library/deep_researcher.py#L435-L509)

In [33]:
# Import the researcher_tools node
from open_deep_library.deep_researcher import researcher_tools

### Node 7: compress_research

**Purpose:** Compresses and synthesizes research findings into a concise, structured summary.

**Key Steps:**
1. Configure compression model
2. Add compression instruction to messages
3. Attempt compression with retry logic:
   - If token limit exceeded, remove older messages
   - Retry up to 3 times
4. Extract raw notes from tool and AI messages
5. Return compressed research and raw notes

**Why Compression?** Researchers may accumulate lots of tool outputs and reflections. Compression ensures:
- All important information is preserved
- Redundant information is deduplicated
- Content stays within token limits for the final report

**Token Limit Handling:** Gracefully handles token limit errors by progressively truncating messages.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 511-585](open_deep_library/deep_researcher.py#L511-L585)

In [34]:
# Import the compress_research node
from open_deep_library.deep_researcher import compress_research

### Node 8: final_report_generation

**Purpose:** Generates the final comprehensive research report from all collected findings.

**Key Steps:**
1. Extract all notes from completed research
2. Configure final report model
3. Attempt report generation with retry logic:
   - If token limit exceeded, truncate findings by 10%
   - Retry up to 3 times
4. Return final report or error message

**Token Limit Strategy:**
- First retry: Use model's token limit × 4 as character limit
- Subsequent retries: Reduce by 10% each time
- Graceful degradation with helpful error messages

**Report Quality:** The prompt guides the model to create well-structured reports with:
- Proper headings and sections
- Inline citations
- Comprehensive coverage of all findings
- Sources section at the end

**Implementation:** [`open_deep_library/deep_researcher.py` lines 607-697](open_deep_library/deep_researcher.py#L607-L697)

In [35]:
# Import the final_report_generation node
from open_deep_library.deep_researcher import final_report_generation

## Task 7: Graph Construction - Putting It All Together

The system is organized into three interconnected graphs:

### 1. Researcher Subgraph (Bottom Level)
Handles individual focused research on a specific topic:
```
START → researcher → researcher_tools → compress_research → END
               ↑            ↓
               └────────────┘ (loops until max iterations or ResearchComplete)
```

### 2. Supervisor Subgraph (Middle Level)
Manages research delegation and coordination:
```
START → supervisor → supervisor_tools → END
            ↑              ↓
            └──────────────┘ (loops until max iterations or ResearchComplete)
            
supervisor_tools spawns multiple researcher_subgraphs in parallel
```

### 3. Main Deep Researcher Graph (Top Level)
Orchestrates the complete research workflow:
```
START → clarify_with_user → write_research_brief → research_supervisor → final_report_generation → END
                 ↓                                       (supervisor_subgraph)
               (may end early if clarification needed)
```

Let's import the compiled graphs from the library.

In [36]:
# Import the pre-compiled graphs from the library
from open_deep_library.deep_researcher import (
    # Bottom level: Individual researcher workflow
    researcher_subgraph,    # Lines 588-605: researcher → researcher_tools → compress_research
    
    # Middle level: Supervisor coordination
    supervisor_subgraph,    # Lines 351-363: supervisor → supervisor_tools (spawns researchers)
    
    # Top level: Complete research workflow
    deep_researcher,        # Lines 699-719: Main graph with all phases
)

## Why This Architecture?

### Advantages of Supervisor-Researcher Delegation

1. **Dynamic Task Decomposition**
   - Unlike section-based approaches with predefined structure, the supervisor can break down research based on the actual question
   - Adapts to different types of research (comparisons, lists, deep dives, etc.)

2. **Parallel Execution**
   - Multiple researchers work simultaneously on different aspects
   - Much faster than sequential section processing
   - Configurable parallelism (1-20 concurrent researchers)

3. **ReAct Pattern for Quality**
   - Researchers use `think_tool` to reflect after each search
   - Prevents excessive searching and improves search quality
   - Natural stopping conditions based on information sufficiency

4. **Flexible Tool Integration**
   - Easy to add MCP tools for specialized research
   - Supports multiple search APIs (Anthropic, Tavily)
   - Each researcher can use different tool combinations

5. **Graceful Token Limit Handling**
   - Compression prevents token overflow
   - Progressive truncation in final report generation
   - Research can scale to arbitrary depths

### Trade-offs

- **Complexity:** More moving parts than section-based approach
- **Cost:** Parallel researchers use more tokens (but faster)
- **Unpredictability:** Research structure emerges dynamically

## Task 8: Running the Deep Researcher

Now let's see the system in action! We'll use it to research wellness strategies for improving sleep quality.

### Setup

We need to:
1. Set up the wellness research request
2. Configure the execution with Anthropic settings
3. Run the research workflow

In [37]:
# Set up the graph with Anthropic configuration
from IPython.display import Markdown, display
import uuid

# Note: deep_researcher is already compiled from the library
# For this demo, we'll use it directly without additional checkpointing
graph = deep_researcher

print("✓ Graph ready for execution")
print("  (Note: The graph is pre-compiled from the library)")

✓ Graph ready for execution
  (Note: The graph is pre-compiled from the library)


### Configuration for Anthropic

We'll configure the system to use:
- **Claude Sonnet 4** for all research, supervision, and report generation
- **Tavily** for web search (you can also use Anthropic's native search)
- **Moderate parallelism** (1 concurrent researcher for cost control)
- **Clarification enabled** (will ask if research scope is unclear)

In [38]:
# Configure for Anthropic with moderate settings
config = {
    "configurable": {
        # Model configuration - using Claude Sonnet 4 for everything
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 10000,
        
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 8192,
        
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 10000,
        
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 8192,
        
        # Research behavior
        "allow_clarification": True,
        "max_concurrent_research_units": 1,  # 1 parallel researcher
        "max_researcher_iterations": 2,      # Supervisor can delegate up to 2 times
        "max_react_tool_calls": 3,           # Each researcher can make up to 3 tool calls
        
        # Search configuration
        "search_api": "tavily",  # Using Tavily for web search
        "max_content_length": 50000,
        
        # Thread ID for this conversation
        "thread_id": str(uuid.uuid4())
    }
}

print("✓ Configuration ready")
print(f"  - Research Model: Claude Sonnet 4")
print(f"  - Max Concurrent Researchers: 1")
print(f"  - Max Iterations: 2")
print(f"  - Search API: Tavily")

✓ Configuration ready
  - Research Model: Claude Sonnet 4
  - Max Concurrent Researchers: 1
  - Max Iterations: 2
  - Search API: Tavily


### Execute the Wellness Research

Now let's run the research! We'll ask the system to research evidence-based strategies for improving sleep quality.

The workflow will:
1. **Clarify** - Check if the request is clear (may skip if obvious)
2. **Research Brief** - Transform our request into a structured brief
3. **Supervisor** - Plan research strategy and delegate to researchers
4. **Parallel Research** - Researchers gather information simultaneously
5. **Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings combined into comprehensive report

In [39]:
# Create our wellness research request
research_request = """
I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please research the best evidence-based strategies for improving sleep quality and create a comprehensive sleep improvement plan for me.
"""

# Execute the graph
async def run_research():
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates"
    ):
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

# Run the research
await run_research()

Starting research workflow...


Node: clarify_with_user

I have sufficient information to proceed with your sleep quality improvement research. I understand you're experiencing:
- Inconsistent bedtime (10pm-1am range)
- Phone use in bed
- Morning fatigue

I will now research evidence-based strategies for improving sleep quality and create a comprehensive sleep improvement plan tailored to address your specific challenges. The research will focus on scientifically-backed methods to establish consistent sleep routines, reduce screen time before bed, and improve overall sleep hygiene to help you wake up feeling more refreshed.

Node: write_research_brief

Research Brief Generated:
I want to improve my sleep quality and need a comprehensive, evidence-based sleep improvement plan. My current sleep challenges include: going to bed at inconsistent times (ranging from 10pm to 1am), using my phone in bed, and often feeling tired in the morning despite getting sleep. Please research the most eff


Node: research_supervisor

Node: final_report_generation

FINAL REPORT GENERATED



Error generating final report: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': "This request would exceed your organization's rate limit of 30,000 input tokens per minute (org: 42a5ec12-e048-4a88-bd86-44072921c3e6, model: claude-sonnet-4-20250514). For details, refer to: https://docs.claude.com/en/api/rate-limits. You can see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase."}, 'request_id': 'req_011CYCixsTXgj77bLfjuHJLV'}


Research workflow completed!


## Task 9: Understanding the Output

Let's break down what happened:

### Phase 1: Clarification
The system checked if your request was clear. Since you provided specific details about your sleep issues, it likely proceeded without asking clarifying questions.

### Phase 2: Research Brief
Your request was transformed into a detailed research brief that guides the supervisor's delegation strategy.

### Phase 3: Supervisor Delegation
The supervisor analyzed the brief and decided how to break down the research:
- Used `think_tool` to plan strategy
- Called `ConductResearch` to delegate to researchers
- Each delegation specified a focused research topic (e.g., sleep hygiene, circadian rhythm, blue light effects)

### Phase 4: Parallel Research
Researchers worked on their assigned topics:
- Each researcher used web search tools to gather information
- Used `think_tool` to reflect after each search
- Decided when they had enough information
- Compressed their findings into clean summaries

### Phase 5: Final Report
All research findings were synthesized into a comprehensive sleep improvement plan with:
- Well-structured sections
- Evidence-based recommendations
- Practical action items
- Sources for further reading

## Task 10: Key Takeaways & Next Steps

### Architecture Benefits
1. **Dynamic Decomposition** - Research structure emerges from the question, not predefined
2. **Parallel Efficiency** - Multiple researchers work simultaneously
3. **ReAct Quality** - Strategic reflection improves search decisions
4. **Scalability** - Handles token limits gracefully through compression
5. **Flexibility** - Easy to add new tools and capabilities

### When to Use This Pattern
- **Complex research questions** that need multi-angle investigation
- **Comparison tasks** where parallel research on different topics is beneficial
- **Open-ended exploration** where structure should emerge dynamically
- **Time-sensitive research** where parallel execution speeds up results

### When to Use Section-Based Instead
- **Highly structured reports** with predefined format requirements
- **Template-based content** where sections are always the same
- **Sequential dependencies** where later sections depend on earlier ones
- **Budget constraints** where token efficiency is critical

### Extend the System
1. **Add MCP Tools** - Integrate specialized tools for your domain
2. **Custom Prompts** - Modify prompts for specific research types
3. **Different Models** - Try different Claude versions or mix models
4. **Persistence** - Use a real database for checkpointing instead of memory

### Learn More
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [Open Deep Research Repo](https://github.com/langchain-ai/open_deep_research)
- [Anthropic Claude Documentation](https://docs.anthropic.com/)
- [Tavily Search API](https://tavily.com/)

## ❓ Question #3:

What are the trade-offs of using parallel researchers vs. sequential research? When might you choose one approach over the other?

##### Answer:
Parallel Research
- Speed is faster
- Cost is higher with higher token consumption
- Context is isolated
- Higer complexity for supervisor to merge the findings

Sequential Research
- Slower
- Less cost and controlled token consumption
- Context aware 
- Lower complexity 

Choose parallel: When time sensitive, compare across different models or distinct topic exploration

Choose sequential: High dependencies, simple fact finding, budget constraints, depth is preferred 

## ❓ Question #4:

How would you adapt this deep research architecture for a production wellness application? What additional components would you need?

##### Answer:
1. Verified Data Retrieval - Beyound web search and Tavily, we should us more authorized medical datasets and databases
2. Persistant state mgmt - Replace in-memory with persistent datastore
3. Medical stafety guardrail - Filter out harmful medical advice or hallucinations
4. Citiation verification - to ground the suggestions and source of truth



## 🏗️ Activity #2: Custom Wellness Research

Using what you've learned, run a custom wellness research task.

**Requirements:**
1. Create a wellness-related research question (exercise, nutrition, stress, etc.)
2. Modify the configuration for your use case
3. Run the research and analyze the output
4. Document what worked well and what could be improved

**Experiment ideas:**
- Research exercise routines for specific conditions (bad knee, lower back pain)
- Compare different stress management techniques
- Investigate nutrition strategies for specific goals
- Explore meditation and mindfulness research

**YOUR CODE HERE**

In [43]:
# YOUR CODE HERE
# Create your own wellness research request and run it

async def run_custom_research(request_content, custom_config):
    """
    Runs the deep research workflow and displays formatted progress for each stage.
    """
    # Initialize the state with the user message
    initial_state = {"messages": [{"role": "user", "content": request_content}]}
    
    # Use graph.astream to catch updates from each node as they complete
    async for event in graph.astream(initial_state, custom_config, stream_mode="updates"):
        for node_name, output in event.items():
            print(f"\n{'='*20} Node: {node_name} {'='*20}")
            
            if node_name == "clarify_with_user":
                msg = output["messages"][-1].content
                print(f"🔍 Clarification Status: {msg}")
            
            elif node_name == "write_research_brief":
                brief = output.get("research_brief", "No brief generated.")
                print(f"Research Brief: {brief[:300]}...")
            
            elif node_name == "supervisor":
                last_msg = output.get("supervisor_messages", [])[-1]
                if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                    print(f"Supervisor is delegating {len(last_msg.tool_calls)} research tasks...")
                    for tc in last_msg.tool_calls:
                        print(f"  - Researching Topic: {tc['args'].get('topic', 'N/A')}")
            
            elif node_name == "supervisor_tools":
                print("Parallel research tasks completed. Gathering findings...")
            
            elif node_name == "final_report_generation":
                print("FINAL REPORT GENERATED \n")
                if "final_report" in output:
                    display(Markdown(output["final_report"]))
                else:
                    print("Error: Report structure found but content missing.")



my_wellness_request = """
    Research the most effective low-impact exercise routines for managing chronic lower back pain in adults.
    Specifically, I would like a comparison between Pilates, Yoga, and Tai Chi.
    Please include evidence-based recommendations for the optimal frequency (sessions per week)
    and duration (minutes per session) for each modality to achieve significant pain reduction.
    """


# Optionally modify the config
my_config = {
    "configurable": {
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 4000,  #lowering to manage token limits
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 8192,
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 10000,
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 8192,
        "allow_clarification": True,
        "max_concurrent_research_units": 1, 
        "max_researcher_iterations": 1, #reducing to 1 to avoid confusion
        "max_react_tool_calls": 2,  
        "search_api": "tavily",
        "max_content_length": 50000,
        "thread_id": str(uuid.uuid4())
    }
}

# Run your research
await run_custom_research(my_wellness_request, my_config)


==================== Node: clarify_with_user ====================
🔍 Clarification Status: I have sufficient information to proceed with your research request. I understand you need a comprehensive comparison of three low-impact exercise modalities (Pilates, Yoga, and Tai Chi) for managing chronic lower back pain in adults. The research will focus on evidence-based effectiveness and include specific recommendations for optimal frequency (sessions per week) and duration (minutes per session) for each modality to achieve significant pain reduction. I will now begin researching the latest scientific evidence and clinical studies to provide you with a thorough analysis.

==================== Node: write_research_brief ====================
Research Brief: I need a comprehensive, evidence-based comparison of three specific low-impact exercise modalities - Pilates, Yoga, and Tai Chi - for managing chronic lower back pain in adults. The research should focus on effectiveness in achieving signi

# Comprehensive Comparison of Low-Impact Exercise Modalities for Chronic Lower Back Pain Management

## Executive Summary

Chronic lower back pain affects millions of adults worldwide and represents one of the leading causes of disability. Low-impact exercise interventions, particularly Pilates, Yoga, and Tai Chi, have emerged as evidence-based therapeutic approaches for managing this condition. This comprehensive analysis examines the scientific literature to provide evidence-based recommendations for optimal implementation of these three modalities in chronic lower back pain management.

## Pilates for Chronic Lower Back Pain

### Effectiveness Evidence

Pilates demonstrates significant effectiveness for chronic lower back pain management, with multiple systematic reviews and randomized controlled trials supporting its therapeutic benefits. A comprehensive systematic review and meta-analysis published in the Journal of Bodywork and Movement Therapies analyzed 14 randomized controlled trials involving 749 participants and found that Pilates exercises resulted in significant improvements in pain intensity and functional disability compared to minimal intervention controls.

The exercise method appears particularly effective due to its focus on core stabilization, spinal alignment, and controlled movement patterns. Clinical studies have consistently shown that Pilates-based interventions lead to improvements in pain scores ranging from 1.5 to 3.2 points on the Visual Analog Scale (0-10), with effect sizes typically ranging from moderate to large (0.5-1.2).

Research published in Physical Therapy demonstrated that mat-based Pilates programs specifically targeting individuals with chronic non-specific low back pain resulted in clinically meaningful improvements in both pain and function. The study followed participants for 6 months and found sustained benefits, suggesting long-term effectiveness of properly implemented Pilates interventions.

### Optimal Frequency and Duration Recommendations

Based on systematic analysis of randomized controlled trials, the optimal Pilates prescription for chronic lower back pain management appears to be:

**Frequency**: 2-3 sessions per week
**Duration**: 45-60 minutes per session
**Program Length**: Minimum 6-8 weeks for initial benefits, with 12-16 weeks recommended for optimal outcomes

A landmark study published in the European Journal of Pain found that participants who engaged in supervised Pilates sessions twice weekly for 60 minutes each session over 12 weeks experienced significantly greater pain reduction compared to those in control groups. The research indicated that sessions shorter than 45 minutes were less effective, while frequencies greater than 3 times per week did not provide additional benefits and increased risk of overexertion.

Clinical research suggests that the initial 4-6 weeks should focus on fundamental movement patterns and core activation, with progression to more complex exercises in subsequent weeks. Studies indicate that supervised sessions, particularly in the initial phases, produce superior outcomes compared to home-based programs alone.

## Yoga for Chronic Lower Back Pain

### Effectiveness Evidence

Yoga has accumulated substantial scientific evidence supporting its effectiveness for chronic lower back pain management. A large-scale systematic review and meta-analysis published in the Cochrane Database of Systematic Reviews examined 12 randomized controlled trials involving 1,080 participants and concluded that yoga is more effective than no exercise and non-exercise controls for reducing pain and improving back-specific function.

The American College of Physicians now includes yoga as a first-line recommended treatment for chronic low back pain in their clinical practice guidelines, based on the strength of available evidence. Research published in the Annals of Internal Medicine demonstrated that yoga participants experienced a 1.5-point greater improvement in pain scores compared to control groups, with benefits persisting at 6-month follow-up assessments.

A particularly notable study published in the Archives of Internal Medicine followed 313 adults with chronic low back pain and found that 12 weeks of yoga classes resulted in superior functional improvements compared to conventional stretching exercises and self-care educational materials. The research demonstrated that yoga's combination of physical postures, breathing techniques, and mindfulness components contributes to its therapeutic effectiveness.

### Optimal Frequency and Duration Recommendations

Evidence-based analysis of clinical trials reveals optimal yoga prescriptions for chronic lower back pain:

**Frequency**: 2-3 sessions per week
**Duration**: 75-90 minutes per session
**Program Length**: 12-16 weeks for optimal benefits, with maintenance sessions thereafter

Research published in Pain Medicine found that participants who attended 90-minute Hatha yoga sessions twice weekly for 12 weeks experienced the most significant improvements in pain intensity and functional capacity. Studies comparing different session durations indicated that 75-90 minute sessions were more effective than shorter 45-60 minute sessions, likely due to the time required for proper warm-up, pose sequences, and relaxation components.

The evidence suggests that Iyengar and Hatha yoga styles demonstrate the strongest research support for chronic lower back pain, with these approaches emphasizing proper alignment, gradual progression, and appropriate modifications for individual limitations. Clinical studies indicate that instructor-led group classes produce superior outcomes compared to home-based video instruction, particularly during the initial learning phases.

## Tai Chi for Chronic Lower Back Pain

### Effectiveness Evidence

Tai Chi demonstrates promising effectiveness for chronic lower back pain management, though the research base is somewhat smaller compared to Pilates and yoga. A systematic review published in the Journal of Pain Research analyzed 7 randomized controlled trials and found that Tai Chi interventions resulted in significant improvements in pain intensity and functional outcomes compared to control conditions.

Research published in the Clinical Journal of Pain examined a 10-week Tai Chi program for adults with chronic low back pain and found significant improvements in pain scores, with participants experiencing an average reduction of 2.1 points on the 0-10 pain scale. The study also demonstrated improvements in balance, flexibility, and quality of life measures.

A notable randomized controlled trial published in Spine followed 160 participants with chronic low back pain through an 18-week Tai Chi intervention. Results showed that Tai Chi participants experienced significant improvements in pain intensity, back-specific function, and overall quality of life compared to wait-list controls. The gentle, flowing movements of Tai Chi appear particularly suitable for individuals with chronic pain who may find more vigorous exercises challenging.

### Optimal Frequency and Duration Recommendations

Based on available clinical research, evidence-based Tai Chi recommendations include:

**Frequency**: 2-3 sessions per week
**Duration**: 60-90 minutes per session
**Program Length**: 12-18 weeks for optimal benefits

Clinical studies suggest that Tai Chi sessions require longer durations compared to other exercise modalities due to the meditative nature of the practice and the time needed for proper warm-up, form instruction, and cool-down periods. Research published in the American Journal of Chinese Medicine found that 90-minute sessions twice weekly over 12 weeks produced superior outcomes compared to shorter or less frequent interventions.

The Yang style of Tai Chi has received the most research attention for chronic lower back pain, with studies indicating that simplified forms (8-24 movements) are as effective as traditional longer forms while being more accessible to beginners and individuals with pain limitations.

## Comparative Analysis and Superior Outcomes

### Direct Comparison Studies

Few studies have directly compared all three modalities within the same research design. However, systematic reviews and meta-analyses allow for indirect comparisons based on effect sizes and clinical outcomes across studies.

**Pain Reduction Effectiveness**:
- Pilates: Effect sizes ranging 0.6-1.2 (moderate to large)
- Yoga: Effect sizes ranging 0.4-0.9 (moderate to large)
- Tai Chi: Effect sizes ranging 0.5-0.8 (moderate)

**Functional Improvement**:
- Pilates shows the strongest evidence for functional improvement, particularly for activities requiring core stability
- Yoga demonstrates broad-spectrum benefits including flexibility, strength, and psychological well-being
- Tai Chi shows particular benefits for balance and coordination alongside pain reduction

### Time to Benefit

Research indicates differential timelines for experiencing benefits:

- **Pilates**: Initial benefits typically observed at 4-6 weeks, with optimal benefits at 8-12 weeks
- **Yoga**: Initial benefits at 4-8 weeks, with continued improvements through 12-16 weeks
- **Tai Chi**: More gradual onset with significant benefits typically emerging at 8-12 weeks

### Long-term Maintenance

Studies examining long-term outcomes suggest:
- Pilates participants show good maintenance of benefits at 6-12 month follow-up
- Yoga demonstrates sustained benefits, particularly when combined with home practice
- Tai Chi shows excellent long-term adherence and maintenance of benefits due to its gentle nature

## Safety Considerations and Contraindications

### General Safety Profile

All three modalities demonstrate excellent safety profiles when properly instructed and progressively implemented. Adverse events are rare and typically minor, consisting of temporary muscle soreness or minor strain.

### Specific Contraindications

**Pilates**:
- Acute spinal injuries or recent surgery (without medical clearance)
- Severe osteoporosis with fracture risk
- Uncontrolled hypertension during intensive sessions
- Pregnancy complications (requires specialized prenatal modifications)

**Yoga**:
- Severe spinal stenosis (certain poses may exacerbate symptoms)
- Recent disc herniation with neurological symptoms
- Uncontrolled blood pressure (particularly with inverted poses)
- Acute inflammatory conditions

**Tai Chi**:
- Severe balance disorders or fall risk without supervision
- Acute joint inflammation
- Uncontrolled cardiovascular conditions
- Severe cognitive impairment affecting safety awareness

### Implementation Recommendations

For all modalities, initial assessment by a healthcare provider is recommended, particularly for individuals with:
- Severe or progressive neurological symptoms
- History of spinal surgery
- Multiple comorbid conditions
- Previous adverse reactions to exercise

Qualified instruction is essential, particularly during initial phases. Instructors should have specific training in working with chronic pain populations and understanding of appropriate modifications.

## Evidence-Based Implementation Guidelines

### Optimal Selection Criteria

**Choose Pilates when**:
- Primary goals include core strengthening and postural improvement
- Individual prefers structured, precise movement patterns
- Functional limitations require targeted strengthening
- Previous positive response to resistance-based exercises

**Choose Yoga when**:
- Comprehensive approach including flexibility, strength, and stress reduction is desired
- Individual is interested in mind-body connection and breathing techniques
- Chronic pain is associated with high stress levels or anxiety
- Preference for varied, flowing movement patterns

**Choose Tai Chi when**:
- Gentle, low-intensity approach is preferred or required
- Balance and coordination improvements are priorities
- Individual has multiple comorbidities limiting exercise options
- Previous negative experiences with higher-intensity exercises

### Progressive Implementation

Regardless of chosen modality, evidence supports a progressive approach:

**Weeks 1-4**: Foundation building with basic movements and proper form instruction
**Weeks 5-8**: Gradual intensity increase with introduction of intermediate techniques
**Weeks 9-12**: Advanced practice with focus on independence and home program development
**Weeks 13+**: Maintenance phase with periodic supervision and program updates

## Conclusion and Clinical Recommendations

Based on comprehensive analysis of available evidence, all three modalities demonstrate significant effectiveness for chronic lower back pain management, with slight variations in specific benefits and optimal implementation protocols.

**Pilates** appears to offer the strongest evidence for functional improvement and core stabilization, with optimal prescription of 2-3 sessions weekly for 45-60 minutes over 12-16 weeks.

**Yoga** provides the most comprehensive approach addressing physical, psychological, and social aspects of chronic pain, with evidence supporting 2-3 sessions weekly for 75-90 minutes over 12-16 weeks.

**Tai Chi** offers the gentlest approach with excellent long-term adherence, optimally prescribed as 2-3 sessions weekly for 60-90 minutes over 12-18 weeks.

Individual selection should be based on personal preferences, physical capabilities, concurrent health conditions, and specific treatment goals. Healthcare providers should consider these factors alongside the evidence presented when making recommendations for chronic lower back pain management through low-impact exercise interventions.

The evidence strongly supports that any of these three modalities, when properly implemented with qualified instruction and appropriate progression, can provide significant benefits for adults with chronic lower back pain. The key to success lies in consistent participation, proper technique, and gradual progression under appropriate supervision.

### Sources

[1] Pilates for chronic non-specific low back pain: Journal of Bodywork and Movement Therapies - https://www.bodyworkmovementtherapies.com/article/S1360-8592(15)00171-4/fulltext

[2] Physical therapy effectiveness of Pilates exercises: Physical Therapy Journal - https://academic.oup.com/ptj/article/95/4/504/2686442

[3] European Journal of Pain Pilates intervention study: European Journal of Pain - https://onlinelibrary.wiley.com/doi/abs/10.1002/ejp.1132

[4] Yoga for chronic low back pain: Cochrane Database Systematic Review - https://www.cochranelibrary.com/cdsr/doi/10.1002/14651858.CD010671.pub2/full

[5] Annals of Internal Medicine yoga effectiveness: Annals of Internal Medicine - https://www.acpjournals.org/doi/10.7326/M16-2579

[6] Archives of Internal Medicine yoga study: Archives of Internal Medicine - https://jamanetwork.com/journals/jamainternalmedicine/fullarticle/414815

[7] Pain Medicine yoga duration study: Pain Medicine Journal - https://academic.oup.com/painmedicine/article/18/11/2138/3098904

[8] Tai Chi for chronic pain systematic review: Journal of Pain Research - https://www.dovepress.com/tai-chi-for-chronic-pain-conditions-a-systematic-review-and-meta-analy-peer-reviewed-fulltext-article-JPR

[9] Clinical Journal of Pain Tai Chi study: Clinical Journal of Pain - https://journals.lww.com/clinicalpain/Abstract/2016/08000/A_Pilot_Randomized_Trial_of_Tai_Chi_for_Chronic.4.aspx

[10] Spine journal Tai Chi intervention: Spine Journal - https://journals.lww.com/spinejournal/Abstract/2011/06150/Tai_Chi_and_Chronic_Pain.15.aspx

[11] American Journal of Chinese Medicine: American Journal of Chinese Medicine - https://www.worldscientific.com/doi/abs/10.1142/S0192415X16500263